In [322]:
import wbgapi as wb
import pandas as pd
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import matplotlib
from datetime import datetime
from dataclasses import dataclass

In [323]:
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [324]:
wme_df = pd.read_csv('world_methane_emission_first_note.csv', index_col = False)

In [325]:
wme_df = wme_df[wme_df['year'] >= 1990]

In [326]:
wme_df = wme_df[wme_df['economy'] == 'CHN'] 

In [327]:
wme_df["future_MethaneValue"] = wme_df["GlobalMethane(ktco2)"].shift(-1)

In [328]:
# wme_df = wme_df[["year", "LandArea(count)", "AirTransport", "GlobalMethane(ktco2)"]]

In [329]:
wme_df = wme_df.set_index('year')

In [330]:
wme_df.shape

(32, 10)

In [313]:
wme_df = wme_df.dropna()

In [314]:
columns = list(wme_df.columns)

In [315]:
columns

['Unnamed: 0',
 'economy',
 'GlobalMethane(ktco2)',
 'LandArea(count)',
 'CO2Emission(kt)',
 'AgricultureMethane(ktco2)',
 'EnergieMethane(ktco2)',
 'AirTransport',
 'note_year',
 'future_MethaneValue']

In [316]:
predictors = columns[2:-1]

In [317]:
predictors

['GlobalMethane(ktco2)',
 'LandArea(count)',
 'CO2Emission(kt)',
 'AgricultureMethane(ktco2)',
 'EnergieMethane(ktco2)',
 'AirTransport',
 'note_year']

In [318]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score
model = RandomForestRegressor(n_estimators=100, min_samples_split=50, random_state=1)
lr = LinearRegression()

train = wme_df.iloc[:-10]
test = wme_df.iloc[-10:]
model.fit(train[predictors], train["future_MethaneValue"])

RandomForestRegressor(min_samples_split=50, random_state=1)

In [319]:
preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
preds

year
2009    854960.733321
2010    854960.733321
2011    854960.733321
2012    854960.733321
2013    854960.733321
2014    854960.733321
2015    854960.733321
2016    854960.733321
2017    854960.733321
2018    854960.733321
dtype: float64

In [320]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [321]:
def backtest(data, model, predictors, start=1095, step=150):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [233]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.6 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/Users/mac/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [235]:
from sklearn.ensemble import GradientBoostingRegressor

In [237]:
est = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0,
    loss='squared_error').fit(train[predictors], train["future_MethaneValue"])